# Задание 1
Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId

In [88]:
import pandas as pd

rating = pd.read_csv('ratings_7.csv')

rating_lifetime = rating.groupby('userId').agg([min, max])
rating_lifetime['lifetime_user'] = (rating_lifetime['timestamp']['max'] - rating_lifetime['timestamp']['min'])/24/3600

for_active_users = rating.groupby('userId').count().filter(items = ['rating'])
active_users = for_active_users [for_active_users ['rating'] > 100]

active_users.join(rating_lifetime, how = 'left').head()



,rating,"(movieId, min)","(movieId, max)","(rating, min)","(rating, max)","(timestamp, min)","(timestamp, max)","(lifetime_user, )"
userId,,,,,,,,
4,204,10,4006,1.0,5.0,949778714,949982274,2.356019
8,116,32,44004,0.5,5.0,1154389340,1154474527,0.985961
15,1700,1,161155,0.5,5.0,997937239,1469330735,5455.943241
17,363,6,34437,0.5,5.0,1127468587,1127476640,0.093206
19,423,1,4970,1.0,5.0,855190091,855195373,0.061134


# Задание 2
Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). Необходимо сформировать две таблицы:
- таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
- аналогичную таблицу по типам выручки с указанием адреса клиента

Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

rzd - железнодорожные перевозки
auto - автомобильные перевозки
air - воздушные перевозки
client_base - адреса клиентов

In [78]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd



,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


In [79]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


In [80]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,173


In [81]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


In [94]:
total_vehicle=rzd.merge(auto, on = 'client_id', how = 'outer').merge(air,on = 'client_id', how = 'outer')
total_vehicle_adress = total_vehicle.merge(client_base, on = 'client_id', how = 'outer')

print(total_vehicle)
print(total_vehicle_adress)

   client_id  rzd_revenue  auto_revenue  air_revenue
0        111       1093.0           NaN          NaN
1        112       2810.0           NaN          NaN
2        113      10283.0       57483.0          NaN
3        114       5774.0          83.0          NaN
4        115        981.0         912.0         81.0
5        116          NaN        4834.0          4.0
6        117          NaN          98.0         13.0
7        118          NaN           NaN        173.0
   client_id  rzd_revenue  auto_revenue  air_revenue          address
0        111       1093.0           NaN          NaN  Комсомольская 4
1        112       2810.0           NaN          NaN   Энтузиастов 8а
2        113      10283.0       57483.0          NaN  Левобережная 1а
3        114       5774.0          83.0          NaN          Мира 14
4        115        981.0         912.0         81.0        ЗЖБИиДК 1
5        116          NaN        4834.0          4.0    Строителей 18
6        117          NaN        

# Задание 3

В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. Т. е. для каждого user_id известна последовательность координат (широта/долгота), когда они требовались приложению для полноценной работы. Как бы вы добавили эти сведения в таблицу визитов и покупок?

Если количество координат для каждого пользователя достаточно велико, то можно их сгруппировать в 3 показателя:
   
 - координаты, когда первый раз "выходит на связь"
 - координаты в которых находится бОльшее количество времени (просматривает страницы, читает отзывы, формирует корзину)
 - координаты, в которых абонент принимает решение и делает заказ.

 Для каждой группы задать определенные интервалы значений координат (не точное значение, а допустимая погрешность).